# Backbone size

This notebook contains analysis of saved number of iterations by using backbones. This can be done simply by taking the ratio of backbone size and controller size.

In [3]:
import pandas as pd
import numpy as np

In [4]:
LOC = "~/Work/Data/FondAsp"
df = pd.read_csv(f"{LOC}/sat_results.csv")
df_backbone = pd.read_csv(f"{LOC}/backbone.csv")
display(df.head())
df_backbone.head()

,Unnamed: 0,domain,instance,planner,type,solved,SAT,time,memory,timeout,memoryout,policysize
0,0,acrobatics,p06,asp1-4h,asp,False,-1,14400.383668,2158.449219,True,False,36
1,1,acrobatics,p06,paladinus-4h,paladinus,True,True,3.153158,82.828125,False,False,128
2,2,acrobatics,p06,minisat-4h,fondsat,False,-1,14400.285015,1983.265625,True,False,-1
3,3,acrobatics,p06,glucose-4h,fondsat,False,-1,14399.936225,610.523438,True,False,-1
4,4,acrobatics,p06,asp2-backbone-4h,asp,False,-1,14400.779252,2543.703125,True,False,38


,domain,instance,backbonesize,time
0,acrobatics,p06,34,0.868
1,acrobatics,p01,5,0.004
2,acrobatics,p03,6,0.016
3,acrobatics,p07,66,3.934
4,acrobatics,p08,100,14.454


### Average number of saved iterations

In [10]:
planner = "asp2-backbone-4h"
df_asp_solved = df.query(f"solved==True and planner==@planner")

minimum_amount = 10
df_asp_counts = df_asp_solved.groupby(["domain", "planner"]).count()[["instance"]]
df_asp_counts.rename(columns={"instance": "solved"}, inplace=True)
df_asp_counts.reset_index(inplace=True)
df_asp_counts.query(f"solved>=@minimum_amount")

,domain,planner,solved
2,blocksworld-ipc08,asp2-backbone-4h,10
5,doors,asp2-backbone-4h,15
6,earth_observation,asp2-backbone-4h,10
8,faults-ipc08,asp2-backbone-4h,45
9,first-responders-ipc08,asp2-backbone-4h,47
10,islands,asp2-backbone-4h,60
11,miner,asp2-backbone-4h,48
12,spiky-tireworld,asp2-backbone-4h,10
13,tireworld,asp2-backbone-4h,11
14,tireworld-truck,asp2-backbone-4h,74


In [8]:
df_all_backbone = df_asp_solved.merge(df_backbone, left_on=["domain", "instance"], right_on=["domain", "instance"], how="inner")
df_all_backbone["gains"] = round(100*df_all_backbone["backbonesize"]/df_all_backbone["policysize"],2)
df_all_backbone

,Unnamed: 0,domain,instance,planner,type,solved,SAT,time_x,memory,timeout,memoryout,policysize,backbonesize,time_y,gains
0,11,acrobatics,p01,asp2-backbone-4h,asp,True,True,0.841773,1.296875,False,False,5,5,0.004,100.00
1,21,acrobatics,p03,asp2-backbone-4h,asp,True,True,6.586345,1.984375,False,False,16,6,0.016,37.50
2,45,acrobatics,p02,asp2-backbone-4h,asp,True,True,0.968556,1.332031,False,False,8,5,0.006,62.50
3,65,acrobatics,p04,asp2-backbone-4h,asp,True,True,512.590000,493.964844,False,False,32,10,0.034,31.25
4,103,blocksworld-ipc08,p06,asp2-backbone-4h,asp,True,True,11.733598,0.742188,False,False,11,9,0.118,81.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,4578,earth_observation,p07,asp2-backbone-4h,asp,True,True,4.568795,1.250000,False,False,17,11,0.022,64.71
359,4602,earth_observation,p08,asp2-backbone-4h,asp,True,True,1.041671,1.066406,False,False,6,5,0.013,83.33
360,4614,earth_observation,p02,asp2-backbone-4h,asp,True,True,1.146394,1.757812,False,False,6,5,0.010,83.33
361,4680,earth_observation,p11,asp2-backbone-4h,asp,True,True,1.173966,1.125000,False,False,7,6,0.012,85.71


In [40]:
df_all = df_all_backbone.loc[:, ~df_all_backbone.columns.str.contains('Unnamed')]



df_all = df_all.drop(columns=["planner", "solved", "type", "SAT", "time_x", "time_y", "memory", "timeout", "memoryout"])
df_all

,domain,instance,policysize,backbonesize,gains
0,triangle-tireworld,p02,16,4,25.00
1,triangle-tireworld,p03,24,6,25.00
2,triangle-tireworld,p04,32,8,25.00
3,triangle-tireworld,p01,8,4,50.00
4,blocksworld-new,p4,8,5,62.50
...,...,...,...,...,...
587,miner,p31,19,10,52.63
588,miner,p07,20,5,25.00
589,miner,p07,20,5,25.00
590,miner,p38,22,8,36.36


In [47]:
df_means = df_all.groupby("domain").mean(["gains", "backbonesize", "policysize"])
df_means

,policysize,backbonesize,gains
domain,,,
acrobatics,15.250000,5.500000,47.342500
beam-walk,18.666667,9.333333,50.000000
blocksworld-ipc08,11.200000,7.200000,64.076000
blocksworld-new,10.500000,7.250000,71.808750
chain-of-rooms,43.000000,28.000000,64.905000
doors,19.000000,9.200000,50.068667
earth_observation,19.200000,10.900000,57.565000
elevators,15.857143,14.571429,91.805714
faults-ipc08,14.933333,7.444444,50.586667


In [49]:
print(df_means.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format))

\begin{tabular}{lrrr}
\toprule
 & policysize & backbonesize & gains \\
domain &  &  &  \\
\midrule
acrobatics & 15.25 & 5.50 & 47.34 \\
beam-walk & 18.67 & 9.33 & 50.00 \\
blocksworld-ipc08 & 11.20 & 7.20 & 64.08 \\
blocksworld-new & 10.50 & 7.25 & 71.81 \\
chain-of-rooms & 43.00 & 28.00 & 64.91 \\
doors & 19.00 & 9.20 & 50.07 \\
earth_observation & 19.20 & 10.90 & 57.56 \\
elevators & 15.86 & 14.57 & 91.81 \\
faults-ipc08 & 14.93 & 7.44 & 50.59 \\
first-responders-ipc08 & 10.57 & 9.53 & 88.35 \\
islands & 8.20 & 4.00 & 54.00 \\
miner & 20.42 & 7.69 & 38.07 \\
spiky-tireworld & 23.00 & 8.00 & 34.78 \\
tireworld & 6.45 & 4.00 & 67.35 \\
tireworld-truck & 15.26 & 4.81 & 32.33 \\
triangle-tireworld & 20.00 & 5.50 & 31.25 \\
zenotravel & 13.40 & 12.40 & 90.78 \\
\bottomrule
\end{tabular}



## Unsat

In [65]:
df_unsat = pd.read_csv(f"{LOC}/unsat_results.csv")
df_unsat_asp = df_unsat.query(f"planner=='asp2-backbone-4h'")

In [67]:
df_large_bb = df_backbone.query("backbonesize>90")
df_unsat_bb = df_large_bb.merge(df_unsat_asp , left_on=["domain", "instance"], right_on=["domain", "instance"], how="inner")
df_unsat_bb.instance.unique()

array(['p109', 'p100', 'p106', 'p19', 'p11', 'p16', 'p20', 'p45', 'p67',
       'p94', 'p56', 'p34', 'p33', 'p50', 'p57', 'p95', 'p66', 'p35',
       'p40', 'p83', 'p79', 'p15', 'p39', 'p99', 'p36'], dtype=object)

In [68]:
len(df_unsat_bb.instance.unique())

25

In [73]:
df_f = df_unsat.query("domain=='first-responders-ipc08'")
df_f.instance.unique()

array(['p109', 'p100', 'p106', 'p19', 'p11', 'p16', 'p20', 'p45', 'p67',
       'p94', 'p56', 'p34', 'p33', 'p50', 'p57', 'p95', 'p66', 'p35',
       'p40', 'p83', 'p79', 'p15', 'p01', 'p39', 'p99', 'p36', 'p09'],
      dtype=object)

In [74]:
len(df_f.instance.unique())

27